In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import re
from wordcloud import WordCloud
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report

# Load dataset
df = pd.read_csv('IMDB Dataset.csv')

# Select first 100 samples
df = df.head(100)

# Convert labels to binary values
df["sentiment"] = df["sentiment"].map({"positive": 1, "negative": 0})

# 1️⃣ Data Distribution (Positive vs Negative Reviews)
plt.figure(figsize=(6, 4))
sns.countplot(x=df["sentiment"].map({1: "Positive", 0: "Negative"}), palette="coolwarm")
plt.title("Distribution of Sentiments")
plt.xlabel("Sentiment")
plt.ylabel("Count")
plt.show()

# Text preprocessing function
def clean_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove punctuation
    return text

df["review"] = df["review"].apply(clean_text)

# 2️⃣ Word Cloud for Most Common Words
text_data = " ".join(df["review"])
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(text_data)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Most Common Words in Reviews")
plt.show()

# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df["review"])
sequences = tokenizer.texts_to_sequences(df["review"])

# Determine appropriate padding length
max_length = int(np.percentile([len(seq) for seq in sequences], 90))
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post")

# Train-Test Split (80-20)
train_size = int(0.8 * len(padded_sequences))
X_train, X_test = padded_sequences[:train_size], padded_sequences[train_size:]
y_train, y_test = df["sentiment"][:train_size], df["sentiment"][train_size:]

# Model Architecture
model = Sequential([
    Embedding(input_dim=10000, output_dim=100, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(32)),
    Dense(16, activation="relu"),
    Dropout(0.2),
    Dense(1, activation="sigmoid")
])

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), 
              loss="binary_crossentropy", 
              metrics=["accuracy"])

# Train model with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=4, callbacks=[early_stopping])

# 3️⃣ Training & Validation Accuracy/Loss Plot
plt.figure(figsize=(12, 5))

# Accuracy Plot
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="Train Accuracy", marker="o")
plt.plot(history.history["val_accuracy"], label="Val Accuracy", marker="o")
plt.title("Training & Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Train Loss", marker="o")
plt.plot(history.history["val_loss"], label="Val Loss", marker="o")
plt.title("Training & Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.show()

# Predictions
predictions = model.predict(X_test)
pred_labels = [1 if p > 0.5 else 0 for p in predictions]

# 4️⃣ Confusion Matrix
conf_matrix = confusion_matrix(y_test, pred_labels)
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Negative", "Positive"], yticklabels=["Negative", "Positive"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# Classification Report
print("Classification Report:\n", classification_report(y_test, pred_labels))

# Print Predictions
for review, pred in zip(df["review"][train_size:], pred_labels):
    sentiment = "positive" if pred == 1 else "negative"
    print(f"Review: {review[:100]}...\nPredicted Sentiment: {sentiment}\n")


In [2]:
from sklearn.metrics import accuracy_score

# Compute Accuracy Score
accuracy = accuracy_score(y_test, pred_labels)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 80.00%
